# Speech Emotion Recognition - Feature Extraction

Databases used

* The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS)
* Toronto emotional speech set (TESS)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
from google.colab import files

Mounted at /content/drive/


### Import Libraries

Import necessary libraries

In [ ]:
import glob
import os
import librosa
import time
import numpy as np
import pandas as pd

### Load all files

We will create our numpy array extracting Mel-frequency cepstral coefficients (MFCCs) while the classes to predict will be extracted from the name of the file.

#### Defining emotions to classify 

Selecting the emotions to be classified. Note that the emotions 'neutral', 'calm' and 'surprised' are only found in RAVDESS dataset and 'pleasantly surprised' or 'ps' is only available in TESS dataset. To combine all the emotions from both datasets into 8 emotion classes, we have changed 'pleasantly surprised' to 'surprised' and 'fearful' to 'fear'

In [ ]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fear',
  '07':'disgust',
  '08':'surprised'
}

#defined tess emotions to test on TESS dataset only
tess_emotions=['angry','disgust','fear','ps','happy','sad']

##defined RAVDESS emotions to test on RAVDESS dataset only
ravdess_emotions=['neutral','calm','angry', 'happy','disgust','sad','fear','surprised']

observed_emotions = ['sad','angry','happy','disgust','surprised','neutral','calm','fear']

#### Feature extraction

Using librosa package we can extract the MFCC features. This function loads the file give the file path and after resampling and computing MFCC features, returns the features. We have selected the no. of MFCCs as 40.

https://librosa.org/librosa/generated/librosa.feature.mfcc.html

In [ ]:
def extract_feature(file_name, mfcc):
    result=[]
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    return result

#### Choosing a dataset

Choose the dataset(s) you want to load using the following function

In [ ]:
def dataset_options():
    # choose datasets
    ravdess = True
    tess = True
    ravdess_speech = False
    ravdess_song = False
    data = {'ravdess':ravdess, 'ravdess_speech':ravdess_speech, 'ravdess_song':ravdess_song, 'tess':tess}
    print(data)
    return data

#### Load data

Load data from the datasets required which is obtained by calling the function dataset__options(). Extract features from each file with the selected emotions in chosen datasets using the extract_feature() function defined.

In [ ]:

def load_data(test_size=0.2): 
    x,y=[],[]
    
    # feature to extract
    mfcc = True
    
    data = dataset_options()
    paths = []
    if data['ravdess']:
        paths.append("/content/drive/MyDrive/Datasets/RAVDESS/*/Actor_*/*.wav")
    elif data['ravdess_speech']:
        paths.append("/content/drive/MyDrive/Datasets/RAVDESS/Speech/Actor_*/*.wav")
    elif data['ravdess_song']:
        paths.append("/content/drive/MyDrive/Datasets/RAVDESS/Song/Actor_*/*.wav")
        
    for path in paths:
        for file in glob.glob(path):
            file_name=os.path.basename(file)
            emotion=emotions[file_name.split("-")[2]] #to get emotion according to filename. dictionary emotions is defined above.
            if emotion not in observed_emotions: #options observed_emotions - RAVDESS and TESS, ravdess_emotions for RAVDESS only
                continue
            feature=extract_feature(file, mfcc)
            x.append(feature)
            y.append(emotion)
            #"..\Datasets\TESS\*AF_*\*.wav"
    if data['tess']:
        for file in glob.glob("/content/drive/MyDrive/Datasets/TESS/*AF_*/*.wav"):
            file_name=os.path.basename(file)
            emotion=file_name.split("_")[2][:-4] #split and remove .wav
            if emotion == 'ps':
                emotion = 'surprised'
            if emotion not in observed_emotions: #options observed_emotions - RAVDESS and TESS, ravdess_emotions for RAVDESS only
                continue
            feature=extract_feature(file, mfcc)
            x.append(feature)
            y.append(emotion)
    return {"X":x,"y":y}

In [ ]:
start_time = time.time()

Trial_dict = load_data(test_size = 0.3)

print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

{'ravdess': True, 'ravdess_speech': False, 'ravdess_song': False, 'tess': True}
--- Data loaded. Loading time: 1674.409150838852 seconds ---


In [ ]:
X = pd.DataFrame(Trial_dict["X"])
y = pd.DataFrame(Trial_dict["y"])

In [ ]:
X.shape, y.shape

((5690, 40), (5690, 1))

In [ ]:
#renaming the label column to emotion
y=y.rename(columns= {0: 'emotion'})

In [ ]:
#concatinating the attributes and label into a single dataframe
data = pd.concat([X, y], axis =1)

In [ ]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,emotion
0,-627.046631,65.364540,-8.032422,10.397017,9.578342,-3.724506,-5.984852,-1.876003,-14.147870,-1.455301,...,-1.966885,-2.954496,-1.910493,-2.532544,-0.590904,-1.193460,-0.400303,-0.709256,0.751061,neutral
1,-641.490479,48.805473,-3.787592,13.445629,2.228184,5.252010,-4.594568,-2.340264,-9.027803,-2.416097,...,-0.108201,-1.965884,-2.345165,-3.696558,-1.473303,-2.498346,-1.452667,-0.622195,-1.455590,sad
2,-688.379822,64.742241,1.362453,14.166891,7.826113,2.353832,-2.716224,1.706661,-7.994302,0.059230,...,-0.821121,-2.257222,-2.710450,-1.786572,0.135882,0.989668,0.331654,-1.502874,-2.535933,calm
3,-655.858582,48.241310,-0.502876,8.743422,0.652527,5.261185,-8.111614,-4.003963,-12.645684,-2.094928,...,-1.871398,-3.000573,-2.197060,-1.652579,0.685930,3.201931,4.583522,5.818406,4.507360,sad
4,-406.141663,25.754354,-20.779034,0.016748,-8.752940,-2.349652,-9.770489,-4.270754,-10.507915,-3.972287,...,0.216467,-1.657515,-0.840737,-1.159853,0.009085,1.541708,0.787645,0.555737,0.312377,angry


## Shuffling data

In [ ]:
#reindexing to shuffle the data at random
data = data.reindex(np.random.permutation(data.index))

In [ ]:
# Storing shuffled ravdess and tess data to avoid loading again
data.to_csv("RAVTESS_MFCC_Observed.csv")
files.download('RAVTESS_MFCC_Observed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>